In [1]:
# Apply under-reporting weights, using factors from 2017 survey analysis
import pandas as pd

In [27]:
# Read in household survey trip records
df = pd.read_excel(r'J:\Projects\Surveys\HHTravel\Survey2014\Data\Final database\Release 4\2014-pr3-M-hhsurvey-trips-LINKED.xlsx',
                  sheetname='Linked trips')

In [28]:
# expwt_final

# Create columns for adjustment factors

# Identify mandatory-non-mandatory
# non-mandatory HB trips are trips that are not from home -> school/work/work-related & school/work/work-related -> home
df['non_mandatory'] = 'mandatory'
_filter = ((df['o_purpose'] == 1) & (df['d_purpose'].isin([2,3,6])) | ((df['d_purpose'] == 1) & df['o_purpose'].isin([2,3,6])))
df.ix[_filter, 'non_mandatory'] = 'non-mandatory'

# Create column for simplified modes
# auto mode includes all hh vehicles tirps, carshare trips, vanpool
df['mode_for_weighting'] = 'other'
df.ix[df['mode'].isin([1,2,3,4,5]), 'mode_for_weighting'] = 'auto'

# bike
df.ix[df['mode'].isin([6]), 'mode_for_weighting'] = 'bike'

# walk
df.ix[df['mode'].isin([7]), 'mode_for_weighting'] = 'walk'

# transit
df.ix[df['mode'].isin([8,9,10,11]), 'mode_for_weighting'] = 'transit'

In [45]:
# Set base weight adjustment to 1
df['wt_adj_factor'] = 1

# Certain trip purposes and modes need an adjustment factpr
# Transit 
_filter = (df['non_mandatory'] == 'non-mandatory') & (df['mode_for_weighting'].isin(['transit','other']))
df.ix[_filter,'wt_adj_factor'] = 1.15

# # Non-mandatory auto trips 0-1 miles
_filter = (df['non_mandatory'] == 'non-mandatory') & (df['mode_for_weighting'].isin(['auto'])) & (df['gdist'] < 1)
df.ix[_filter,'wt_adj_factor'] = 2

# # Non-mandatory auto trips > 1 mile
_filter = (df['non_mandatory'] == 'non-mandatory') & (df['mode_for_weighting'].isin(['auto'])) & (df['gdist'] >= 1)
df.ix[_filter,'wt_adj_factor'] = 1.15

In [49]:
# Multiple the existing weights by this adjustment factor to get a new adjusted weight
df['trip_wt_final'] = df['expwt_final']*df['wt_adj_factor']

In [58]:
# Load person file to get total population using hh/person weights
person = pd.read_excel(r'J:\Projects\Surveys\HHTravel\Survey2014\Data\Final database\Release 4\2014-pr3-M-hhsurvey-persons.xlsx',
                      sheetname='Data1')

In [65]:
tot_person = person['expwt_final'].sum()
tot_person

3464535.644400008

In [ ]:
# Get total person days
person_days = pd.read_excel(r'J:\Projects\Surveys\HHTravel\Survey2014\Data\Final database\Release 4\2014-pr3-M-hhsurvey-persons.xlsx',
                      sheetname='Data1')

In [60]:
df['trip_wt_final'].sum()/tot_person

4.009771256741059

In [61]:
df['expwt_final'].sum()/tot_person

3.8392630489744968

In [53]:
# Write results to file
df.to_excel(r'trips_2014_adjusted_wt.xlsx', index=False)

In [64]:
df['expwt_final'].sum()

13301263.681399997

In [68]:
# For 2017 survey, person-day totals
pd17 = pd.read_excel(r'J:\Projects\Surveys\HHTravel\Survey2017\Data\Dataset from RSG 180307\person_day_level_weights.xlsx',
                     sheetname='person_day_level_weights')

In [95]:
# pd17

In [70]:
pd17['hh_wt_final'].sum()

4897102.927818465

In [72]:
pd17['hh_day_wt_final'].sum()

3850151.9588035243

In [74]:
pd17['hh_wt_adjusted'].sum()

4807970.040778006

In [77]:
pd17['hh_day_wt_adjusted'].sum()

3781685.4633519715

In [96]:
# Person weights
ps17 = pd.read_excel(r'J:\Projects\Surveys\HHTravel\Survey2017\Data\Dataset from RSG 180307\person_level_weights.xlsx',
                     sheetname='person_level_weights')

In [98]:
ps17['hh_wt_final'].sum()

3850151.958803499

In [100]:
ps17['hh_day_wt_final'].sum()

3533789.234364999

In [102]:
ps17['hh_wt_adjusted'].sum()

3781685.463351941

In [104]:
ps17['hh_day_wt_adjusted'].sum()

3476452.931693045

In [88]:
# GPS Weighted 2014 data
gps14_person = pd.read_csv(r'R:\SoundCastDocuments\2014Estimation\Files_From_Mark_2014\gps_weights_11_3_16\original\precP14_w.dat',
                           delim_whitespace=True)

In [90]:
gps14_person['psexpfac'].sum()

3708517.966599983

In [91]:
# GPS Weighted 2014 data
gps14_hh = pd.read_csv(r'R:\SoundCastDocuments\2014Estimation\Files_From_Mark_2014\gps_weights_11_3_16\original\hrecP14_w.dat',
                           delim_whitespace=True)
gps14_hh['hhexpfac'].sum()

1530318.962199996

In [92]:
# GPS Weighted 2014 data
gps14_trip = pd.read_csv(r'R:\SoundCastDocuments\2014Estimation\Files_From_Mark_2014\gps_weights_11_3_16\original\tripP14_w.dat',
                           delim_whitespace=True)

In [93]:
gps14_trip['trexpfac'].sum()

16842635.243592683

In [105]:
gps14_pday = pd.read_csv(r'R:\SoundCastDocuments\2014Estimation\Files_From_Mark_2014\gps_weights_11_3_16\original\pdayP14_w.dat',
                           delim_whitespace=True)

In [107]:
gps14_pday['pdexpfac'].sum()

3708517.9658475714